In [2]:
import jsonlines
import os
import numpy as np
import re
import pyrouge

In [3]:
# get the summarization based on first three sentences of the content
# and save them as a story/txt file with the same name(?)

# choose a subreddit
input_subreddit = "AskReddit"

# read json dataset
path = "/home/ubuntu/cs224u/raw_reddit/" + "tldr-training-data.jsonl"
reader = jsonlines.open(path)

In [4]:
cwd = os.getcwd()
cwd

'/home/ubuntu/cs224u'

In [5]:
# create the directory to the corresponding
make_dir = cwd + '/'+input_subreddit+'_baseline' 
os.mkdir(make_dir)

OSError: [Errno 17] File exists: '/home/ubuntu/cs224u/AskReddit_baseline_summarization'

In [88]:
make_dir

'/home/ubuntu/cs224u/AskReddit_baseline_summarization'

In [89]:
make_dec_dir = make_dir + '/decoded'
os.mkdir(make_dec_dir)

make_ref_dir = make_dir + '/reference'
os.mkdir(make_ref_dir)

In [6]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [7]:
# create a .txt file and save it to the directory save_dir using baseline summarization method
# (first three sentence):
def create_baseline_summarization_file(input_dict, dec_dir):
    '''
    input:
    input_dict: input dictionary, include information about its id, content, summary etc
    dec_dir: a directory where to save the summarization files
    '''
    content = input_dict["content"]
    #print(content)
    #print("\n")
    summarization_top3 = ' '.join(re.split(r'(?<=[.:;])\s', content)[:3])
    #print(summarization_top3)
    dic_id = input_dict["id"]
    filename = os.path.join(dec_dir, dic_id + "_decoded.txt")
    file1 = open(filename,"w")
    file1.writelines(summarization_top3)
    file1.close()

In [8]:
# create a .txt file and save it to the directory save_dir using baseline summarization method
# (first three sentence):
def create_reference_file(input_dict, ref_dir):
    '''
    input:
    input_dict: input dictionary, include information about its id, content, summary etc
    ref_dir: a directory about where to save the summarization files
    '''
    reference = input_dict["summary"].encode('utf-8')
    #print(type(reference))
    #content = input_dict["content"]
    #summarization_with_first_3_sentences = ' '.join(re.split(r'(?<=[.:;])\s', content)[:3])
    dic_id = input_dict["id"]
    filename = os.path.join(ref_dir, dic_id + "_reference.txt")
    file1 = open(filename,"w")
    file1.writelines(reference)
    file1.close()

In [93]:
# create corresponding baseline summarization
for dic in reader:
    if("subreddit" in dic.keys() and dic["subreddit"] == input_subreddit and isEnglish(dic["content"]) == True ):
        create_baseline_summarization_file(dic, make_dec_dir)
        create_reference_file(dic, make_ref_dir)
        

In [95]:
# create the directory to the corresponding
make_dir_baseline_summarization_test = cwd + '/'+input_subreddit+'_baseline_summarization_test'
os.mkdir(make_dir_baseline_summarization_test)
make_dec_baseline_dir = make_dir_baseline_summarization_test + '/decoded'
os.mkdir(make_dec_dir)

make_ref_baseline_dir = make_dir_make_dir_baseline_summarization_test + '/reference'
os.mkdir(make_ref_dir)

OSError: [Errno 17] File exists: '/home/ubuntu/cs224u/AskReddit_baseline_summarization/decoded'

In [1]:
test_list_raw = cwd+'/processed_ask/AskReddit_testlist.txt'

NameError: name 'cwd' is not defined

In [ ]:
# create corresponding baseline summarization
for dic in reader:
    if("subreddit" in dic.keys() and dic["subreddit"] == input_subreddit and isEnglish(dic["content"]) == True and dic.id in test_list):
        create_baseline_summarization_file(dic, make_dec_dir)
        create_reference_file(dic, make_ref_dir)

In [94]:
from pyrouge import Rouge155

In [77]:
# compute the rouge score
def rouge_eval(ref_dir, dec_dir):
  """Evaluate the files in ref_dir and dec_dir with pyrouge, returning results_dict"""
  r = pyrouge.Rouge155()
  r.model_filename_pattern = '#ID#_reference.txt'
  r.system_filename_pattern = '(\d+)_decoded.txt'
  r.model_dir = ref_dir
  r.system_dir = dec_dir
  logging.getLogger('global').setLevel(logging.WARNING) # silence pyrouge logging
  rouge_results = r.convert_and_evaluate()
  return r.output_to_dict(rouge_results)

In [80]:
result = rouge_eval(make_ref_dir, make_dec_dir)

IOError: [Errno 2] No such file or directory: u'/root/.pyrouge/settings.ini'

In [76]:
def rouge_log(results_dict, dir_to_write):
  """Log ROUGE results to screen and write to file.

  Args:
    results_dict: the dictionary returned by pyrouge
    dir_to_write: the directory where we will write the results to"""
  log_str = ""
  for x in ["1","2","l"]:
    log_str += "\nROUGE-%s:\n" % x
    for y in ["f_score", "recall", "precision"]:
      key = "rouge_%s_%s" % (x,y)
      key_cb = key + "_cb"
      key_ce = key + "_ce"
      val = results_dict[key]
      val_cb = results_dict[key_cb]
      val_ce = results_dict[key_ce]
      log_str += "%s: %.4f with confidence interval (%.4f, %.4f)\n" % (key, val, val_cb, val_ce)
  tf.logging.info(log_str) # log to screen
  results_file = os.path.join(dir_to_write, "ROUGE_results.txt")
  tf.logging.info("Writing final ROUGE results to %s...", results_file)
  with open(results_file, "w") as f:
    f.write(log_str)